In [1]:
import os

from IPython import display
# import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random as python_random

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import regularizers

from lazypredict.Supervised import LazyClassifier

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


yamnet_model = hub.load('https://tfhub.dev/google/yamnet/1') # if there's an "SavedModel file does not exist at:", delete that folder and rerun it
vggish_model = hub.load('https://tfhub.dev/google/vggish/1')

2023-03-05 23:13:51.794808: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


OSError: SavedModel file does not exist at: /var/folders/fw/slnm06yn15lgwlcg_7jpt3t00000gn/T/tfhub_modules/9616fd04ec2360621642ef9455b84f4b668e219e/{saved_model.pbtxt|saved_model.pb}

In [ ]:
df_all = pd.read_csv('../train_val_test_split/train_val_test_GoogleAudioSet.csv', index_col=0)
df_all

In [ ]:
def data_preprocessing(filename):
    import pickle
    
    file = open(filename, 'rb')
    output = pickle.load(file)
#     output = pd.read_pickle(file)
    wav_raw = output['y']
    wav_bg = output['bg_y']
    wav_fg = output['fg_y']
    
    df_indices_raw = output['df_indices']
    df_indices_fg = output['df_indices_fg']
    df_indices_bg = output['df_indices_bg']
    
    wt = output['wt']
    mps_raw = output['mps']
    mps_fg = output['mps_fg']
    mps_bg = output['mps_bg']
    mps_raw = mps_raw[:,wt<=100].reshape(-1) # exclude the temporal modulation frequency >100 Hz
    mps_fg = mps_fg[:,wt<=100].reshape(-1)
    mps_bg = mps_bg[:,wt<=100].reshape(-1)
    
    file.close()

    # Extract YAMNet embeddings for each frame
    scores, embedding_tensor_raw, spectrogram = yamnet_model(wav_raw)
    embedding_tensor_raw = tf.reduce_mean(embedding_tensor_raw, axis=0).numpy()

    scores, embedding_tensor_bg, spectrogram = yamnet_model(wav_bg)
    embedding_tensor_bg = tf.reduce_mean(embedding_tensor_bg, axis=0).numpy()

    scores, embedding_tensor_fg, spectrogram = yamnet_model(wav_fg)
    embedding_tensor_fg = tf.reduce_mean(embedding_tensor_fg, axis=0).numpy()
    
    
    # Extract VGGish embeddings for each frame
    vggish_embedding_raw = tf.reduce_mean(vggish_model(wav_raw), axis=0).numpy()
    vggish_embedding_bg = tf.reduce_mean(vggish_model(wav_bg), axis=0).numpy()
    vggish_embedding_fg = tf.reduce_mean(vggish_model(wav_fg), axis=0).numpy()
   
    return embedding_tensor_raw, embedding_tensor_bg, embedding_tensor_fg, mps_raw, mps_bg, mps_fg, df_indices_raw, df_indices_bg, df_indices_fg, vggish_embedding_raw, vggish_embedding_bg, vggish_embedding_fg

In [ ]:
embedding_raw_list = []
embedding_bg_list = []
embedding_fg_list = []
mps_raw_list = []
mps_bg_list = []
mps_fg_list = []
indices_raw_list = []
indices_bg_list = []
indices_fg_list = []
vgg_raw_list = []
vgg_bg_list = []
vgg_fg_list = []

import time
start_time = time.time()


for index, row in df_all.iterrows():
    embedding_tensor_raw, embedding_tensor_bg, embedding_tensor_fg, mps_raw, mps_bg, mps_fg, df_indices_raw, df_indices_bg, df_indices_fg, vggish_embedding_raw, vggish_embedding_bg, vggish_embedding_fg = data_preprocessing(row['file'])
    
    embedding_raw_list.append(embedding_tensor_raw)
    embedding_bg_list.append(embedding_tensor_bg)
    embedding_fg_list.append(embedding_tensor_fg)
    mps_raw_list.append(mps_raw)
    mps_bg_list.append(mps_bg)
    mps_fg_list.append(mps_fg)
    indices_raw_list.append(df_indices_raw)
    indices_bg_list.append(df_indices_bg)
    indices_fg_list.append(df_indices_fg)
    vgg_raw_list.append(vggish_embedding_raw)
    vgg_bg_list.append(vggish_embedding_bg)
    vgg_fg_list.append(vggish_embedding_fg)

    
print('seconds: '+str(time.time()-start_time))


In [ ]:
embedding_raw_matrix = np.stack(embedding_raw_list, axis=0)
embedding_bg_matrix = np.stack(embedding_bg_list, axis=0)
embedding_fg_matrix = np.stack(embedding_fg_list, axis=0)

In [ ]:
df_indices_raw = pd.concat(indices_raw_list, ignore_index=True)
df_indices_bg = pd.concat(indices_bg_list, ignore_index=True)
df_indices_fg = pd.concat(indices_fg_list, ignore_index=True)

In [ ]:
mps_raw_matrix = np.stack(mps_raw_list)
mps_bg_matrix = np.stack(mps_bg_list)
mps_fg_matrix = np.stack(mps_fg_list)

In [ ]:
vgg_raw_matrix = np.stack(vgg_raw_list, axis=0)
vgg_bg_matrix = np.stack(vgg_bg_list, axis=0)
vgg_fg_matrix = np.stack(vgg_fg_list, axis=0)

In [ ]:
np.save('../data/processed/20230304/embedding_raw_matrix.npy', embedding_raw_matrix)
np.save('../data/processed/20230304/embedding_bg_matrix.npy', embedding_bg_matrix)
np.save('../data/processed/20230304/embedding_fg_matrix.npy', embedding_fg_matrix)

np.save('../data/processed/20230304/vgg_raw_matrix.npy', vgg_raw_matrix)
np.save('../data/processed/20230304/vgg_bg_matrix.npy', vgg_bg_matrix)
np.save('../data/processed/20230304/vgg_fg_matrix.npy', vgg_fg_matrix)

np.save('../data/processed/20230304/mps_raw_matrix.npy', mps_raw_matrix)
np.save('../data/processed/20230304/mps_bg_matrix.npy', mps_bg_matrix)
np.save('../data/processed/20230304/mps_fg_matrix.npy', mps_fg_matrix)

df_indices_raw.to_csv('../data/processed/20230304/df_indices_raw.csv')
df_indices_bg.to_csv('../data/processed/20230304/df_indices_bg.csv')
df_indices_fg.to_csv('../data/processed/20230304/df_indices_fg.csv')